# Development Right Transfers
#### Meeting Agenda: Development Rights Transfer & Conversion Analysis

1. Objectives & Data Review
    * Goal: Standardize ETL of transfer → conversion process & confirm data integration
    * Review of data sources:
        * LT Info TDR Transactions (APN, type, land capability, quantity)
        * Parcel Master (jurisdiction, town center proximity, local plan)
        * Accela (transfer status, permit data)
2. Coding Plan & Standardization
    * Walkthrough of data integration approach
    * Confirm transfer first, then convert process
    * Address any inconsistencies or edge cases
3. Key Analyses
    * Land Capability: SEZ, sensitive, non-sensitive
    * Distance from Center: Trends by proximity
    * Interjurisdictional Activity: Transfers & conversions across boundaries
4. Next Steps
    * Assign action items & timeline for completion


## Setup

### Packages

In [ ]:
import pandas as pd
import os
import pathlib
import arcpy
from arcgis.features import FeatureLayer, GeoAccessor, GeoSeriesAccessor
from utils import *
from datetime import datetime
from time import strftime  

### Global Variables

In [ ]:
# set data frame display options
# pandas options
pd.options.mode.copy_on_write = True
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows    = 999
pd.options.display.float_format = '{:,.2f}'.format
   
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)
arcpy.env.outputCoordinateSystem = sr

# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir   = local_path.parents[0] / 'Reporting/data/raw_data'
# folder to save processed data
out_dir    = local_path.parents[0] / 'Reporting/data/processed_data'

# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")
sdeEdit    = os.path.join(filePath, "Edit.sde")

## Data Processing
### Data Pipeline Overview
1. Extract data from LT Info, Parcel Master, and Accela.
2. Clean and preprocess data for consistency.
3. Merge datasets using APN as the primary key.
4. Standardize workflow: **transfer first, then convert**.
5. Identify and resolve inconsistencies.

### 1. Extract data from LT Info, Parcel Master, and Accela.

#### Data Sources
- **LT Info TDR Transactions**: Tracks APN, development right type, land capability, and quantity.
- **Parcel Master**: Provides jurisdiction, town center proximity, and
- **Accela**: Contains transfer status and permit details.

> Sources
* https://www.laketahoeinfo.org/WebServices/List
* https://maps.trpa.org/server/rest/services/
* https://parcels.laketahoeinfo.org/TdrTransaction/TransactionList
* sdeBase, sdeCollect, sdeTabular

#### 1.1 Parcel Master

In [ ]:
# web service and database paths
# portal_ParcelMaster = 'https://maps.trpa.org/server/rest/services/Parcel_Master/FeatureServer/0'
sde_ParcelMaster    = os.path.join(sdeBase,"\\sde.SDE.Parcels\\sde.SDE.Parcel_Master")

# get spatially enabled dataframes
sdfParcels = pd.DataFrame.spatial.from_featureclass(sde_ParcelMaster)

#### 1.2 LTInfo Data

In [ ]:
# transfer grid downloaded from LTinfo https://parcels.laketahoeinfo.org/TdrTransaction/TransactionList
dfTransactionsGrid = pd.read_csv(local_path/"data\raw_data\TransactedAndBankedDevelopmentRights.csv")

In [ ]:
## LT Info Data
# Verified Development Rights from Accela as a DataFrame
dfDevRightForAccela  = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelDevelopmentRightsForAccela/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# Development Rights Transacted and Banked as a DataFrame
dfDevRightTransacted = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# All Parcels as a DataFrame
dfLTParcel           = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

#### 1.3 Accela Permit Data

In [ ]:
# API access to download excel file of Accela Record Details
accelaRecorDetails = "https://laketahoeinfo.org/Api/GetAccelaRecordDetailsExcel/1A77D078-B83E-44E0-8CA5-8D7429E1A6B4"
# download the file
dfAccelaRecord = pd.read_excel(accelaRecorDetails)

### 2. Clean and preprocess data for consistency.

In [ ]:
# get 2024 data

dfPermit24 = dfAccelaRecord[dfAccelaRecord['File Date'].dt.year == 2024]


In [ ]:
dfPermit24['Accela CAPType Name'].value_counts()

In [ ]:
# get 2024 data for Building/ERS/Admin/Transfer Of Development
dfPermit24_Transfers = dfPermit24[dfPermit24['Accela CAPType Name'] == 'Building/ERS/Admin/Transfer Of Development']
dfPermit24_Transfers

### 3. Merge datasets using APN as the primary key.

### 4. Standardize workflow: **transfer first, then convert**.

### 5. Identify and resolve inconsistencies.

## Key Analyses & Insights

### Land Capability Analysis
- Categorize transfers by SEZ, sensitive, and non-sensitive land.
### Proximity Analysis
- Assess distance of transfers from town centers.
### Interjurisdictional Transfers
- Examine development right transfers across jurisdictional boundaries.

## Next Steps
### Action Items
- 